In [12]:
from pathlib import Path
import pandas as pd
from skimage.feature import peak_local_max
import mrcfile
import napari
import starfile

In [15]:
tomo_path = Path("/media/michalakdj/scratch/eman2_models/tomograms/ts002_corrected.mrc")
tomo = mrcfile.read(tomo_path)
viewer = napari.view_image(tomo)

In [16]:
segment_path = Path("../eman2_models/segmentations/ts002_corrected__ribosome_good_seg.mrc")
segment_map = mrcfile.read(segment_path)
viewer.add_image(segment_map, colormap="gist_earth")

<Image layer 'segment_map' at 0x7f9d410ac0d0>

In [ ]:
threshold_rel = 0.1
peak_coords = peak_local_max(segment_map, min_distance=15, threshold_rel=threshold_rel, ) # coordinates at (z,y,x)
viewer.add_points(peak_coords, name=f"Peaks > {threshold_rel}")

<Points layer 'Peaks >0.1' at 0x7f9a9937f430>

  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
    self.io_loop.start()
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/home/michalakdj/mambaforge/envs/napari/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

In [4]:
tomo_bin_factor = 10

peak_coords_df = pd.DataFrame(peak_coords,columns=["z","y","x"])
unbinned_peak_coords_df = pd.DataFrame()
unbinned_peak_coords_df["x"] = tomo_bin_factor * peak_coords_df["x"]
unbinned_peak_coords_df["y"] = tomo_bin_factor * peak_coords_df["y"]
unbinned_peak_coords_df["z"] = tomo_bin_factor * peak_coords_df["z"]
coord_dict = {"x" : "rlnCoordinateX", "y" : "rlnCoordinateY", "z" : "rlnCoordinateZ"}
unbinned_peak_coords_df.rename(mapper=coord_dict, axis=1, inplace=True)
unbinned_peak_coords_df["rlnAngleRot"] = 0.0
unbinned_peak_coords_df["rlnAngleTilt"] = 0.0
unbinned_peak_coords_df["rlnAnglePsi"] = 0.0
unbinned_peak_coords_df["rlnTomoName"] = "ts002"

In [5]:
unbinned_peak_coords_df

,rlnCoordinateX,rlnCoordinateY,rlnCoordinateZ,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnTomoName
0,1240,3860,2240,0.0,0.0,0.0,ts002
1,1340,4880,2200,0.0,0.0,0.0,ts002
2,7060,7160,2390,0.0,0.0,0.0,ts002
3,2720,1160,2420,0.0,0.0,0.0,ts002
4,2440,9840,3150,0.0,0.0,0.0,ts002
...,...,...,...,...,...,...,...
212,240,4480,2170,0.0,0.0,0.0,ts002
213,1520,220,1730,0.0,0.0,0.0,ts002
214,120,3260,2220,0.0,0.0,0.0,ts002
215,8200,8620,1970,0.0,0.0,0.0,ts002


In [7]:
write_path = Path("/media/michalakdj/scratch/eman2_test/segmentations/ribosome_ts002.star")
starfile.write(unbinned_peak_coords_df, write_path, overwrite=True)